In [4]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Documents/Projects/pfdelta/data_generation`


In [33]:
import PowerModels
import PowerModels: ref, var, ids
const PM = PowerModels
import JuMP
import Ipopt
include("src/build_opf_pfdelta.jl")

variable_branch_power_pfdelta (generic function with 1 method)

In [34]:
net = PM.parse_matpower("pglib/pglib_opf_case5_pjm.m")

[info | PowerModels]: extending matpower format with data: areas 1x3
[info | PowerModels]: removing 1 cost terms from generator 4: [4000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [1400.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [1500.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [3000.0, 0.0]


Dict{String, Any} with 14 entries:
  "bus"            => Dict{String, Any}("4"=>Dict{String, Any}("zone"=>1, "bus_…
  "source_type"    => "matpower"
  "name"           => "pglib_opf_case5_pjm"
  "dcline"         => Dict{String, Any}()
  "source_version" => "2"
  "gen"            => Dict{String, Any}("4"=>Dict{String, Any}("pg"=>1.0, "mode…
  "branch"         => Dict{String, Any}("4"=>Dict{String, Any}("br_r"=>0.00108,…
  "storage"        => Dict{String, Any}()
  "switch"         => Dict{String, Any}()
  "baseMVA"        => 100.0
  "per_unit"       => true
  "areas"          => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>Any…
  "shunt"          => Dict{String, Any}()
  "load"           => Dict{String, Any}("1"=>Dict{String, Any}("source_id"=>Any…

In [35]:
pm = PM.instantiate_model(net, PM.ACPPowerModel, build_opf_power_flow_delta)
print(pm.model)

Min 1000 0_pg[5] + 4000 0_pg[4] + 1500 0_pg[2] + 3000 0_pg[3] + 1400 0_pg[1]
Subject to
 0_va[4] = 0.0
 -0_pg[5] + 0_p[(6, 5, 4)] + 0_p[(3, 5, 1)] = 0.0
 -0_qg[5] + 0_q[(6, 5, 4)] + 0_q[(3, 5, 1)] = 0.0
 -0_pg[4] + 0_p[(6, 4, 5)] + 0_p[(5, 4, 3)] + 0_p[(2, 4, 1)] = -4.0
 -0_qg[4] + 0_q[(6, 4, 5)] + 0_q[(5, 4, 3)] + 0_q[(2, 4, 1)] = -1.3147
 0_p[(4, 2, 3)] + 0_p[(1, 2, 1)] = -3.0
 0_q[(4, 2, 3)] + 0_q[(1, 2, 1)] = -0.9861
 -0_pg[3] + 0_p[(5, 3, 4)] + 0_p[(4, 3, 2)] = -3.0
 -0_qg[3] + 0_q[(5, 3, 4)] + 0_q[(4, 3, 2)] = -0.9861
 -0_pg[2] - 0_pg[1] + 0_p[(2, 1, 4)] + 0_p[(3, 1, 5)] + 0_p[(1, 1, 2)] = 0.0
 -0_qg[2] - 0_qg[1] + 0_q[(2, 1, 4)] + 0_q[(3, 1, 5)] + 0_q[(1, 1, 2)] = 0.0
 0_vm[5] ≥ 0.9
 0_vm[4] ≥ 0.9
 0_vm[3] ≥ 0.9
 0_vm[1] ≥ 0.9
 0_pg[5] ≥ 6.0
 0_pg[2] ≥ 1.7
 0_pg[3] ≥ 5.2
 0_pg[1] ≥ 0.4
 0_vm[5] ≤ 1.1
 0_vm[4] ≤ 1.1
 0_vm[3] ≤ 1.1
 0_vm[1] ≤ 1.1
 0_p[(5, 3, 4)] - (((3.3336667000033335 + 0.0) / 1.0 ^ 2.0) * 0_vm[3] ^ 2.0 + ((-3.3336667000033335 * 1.0 + -33.33666700003334 * 0.0) / 

In [37]:
result = PM.optimize_model!(pm, optimizer=Ipopt.Optimizer)

UndefVarError: UndefVarError: `libipopt` not defined in `Ipopt`
Suggestion: check for spelling errors or missing imports.

In [ ]:
results

In [19]:
pm_opf = PM.instantiate_model(net, PM.ACPPowerModel, PM.build_opf)
print(pm_opf.model)

Min 1000 0_pg[5] + 4000 0_pg[4] + 1500 0_pg[2] + 3000 0_pg[3] + 1400 0_pg[1]
Subject to
 0_va[4] = 0.0
 -0_pg[5] + 0_p[(6, 5, 4)] + 0_p[(3, 5, 1)] = 0.0
 -0_qg[5] + 0_q[(6, 5, 4)] + 0_q[(3, 5, 1)] = 0.0
 -0_pg[4] + 0_p[(6, 4, 5)] + 0_p[(5, 4, 3)] + 0_p[(2, 4, 1)] = -4.0
 -0_qg[4] + 0_q[(6, 4, 5)] + 0_q[(5, 4, 3)] + 0_q[(2, 4, 1)] = -1.3147
 0_p[(4, 2, 3)] + 0_p[(1, 2, 1)] = -3.0
 0_q[(4, 2, 3)] + 0_q[(1, 2, 1)] = -0.9861
 -0_pg[3] + 0_p[(5, 3, 4)] + 0_p[(4, 3, 2)] = -3.0
 -0_qg[3] + 0_q[(5, 3, 4)] + 0_q[(4, 3, 2)] = -0.9861
 -0_pg[2] - 0_pg[1] + 0_p[(2, 1, 4)] + 0_p[(3, 1, 5)] + 0_p[(1, 1, 2)] = 0.0
 -0_qg[2] - 0_qg[1] + 0_q[(2, 1, 4)] + 0_q[(3, 1, 5)] + 0_q[(1, 1, 2)] = 0.0
 -0_va[4] + 0_va[3] ≥ -0.5235987755982988
 0_va[2] - 0_va[3] ≥ -0.5235987755982988
 -0_va[5] + 0_va[4] ≥ -0.5235987755982988
 -0_va[4] + 0_va[1] ≥ -0.5235987755982988
 -0_va[5] + 0_va[1] ≥ -0.5235987755982988
 -0_va[2] + 0_va[1] ≥ -0.5235987755982988
 -0_va[4] + 0_va[3] ≤ 0.5235987755982988
 0_va[2] - 0_va[3] ≤ 0.5